# Data_Generator

In [1]:
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit, execute, Aer
import pennylane as qml
import numpy as np
from numpy import pi
simulator = Aer.get_backend('qasm_simulator')



# Ansatz

In [9]:
def Operational_circuit(params):
    qc = QuantumCircuit(1)
    
    #qc.h(range(2))
    qc.ry(params[0],0)
    
    return qc
    

In [21]:
Operational_circuit_try = Operational_circuit([0])

In [22]:
Operational_circuit_try.draw()

┌───────┐
q_0: ┤ RY(0) ├
     └───────┘

# Measurments for different components of H

# 1. Z

In [23]:
def measure_z_circuit(given_circuit):
    z_meas = given_circuit.copy()
    z_meas.measure_all()
    return z_meas

z_meas = measure_z_circuit(Operational_circuit_try)
z_meas.draw()

┌───────┐ ░ ┌─┐
   q_0: ┤ RY(0) ├─░─┤M├
        └───────┘ ░ └╥┘
meas_0: ═════════════╩═

In [24]:
def measure_z(given_circuit, num_shots = 10000):

    z_meas = measure_z_circuit(given_circuit)
    
    result = execute(z_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(z_meas)

    if '0' not in counts:
        counts['0'] = 0
    if '1' not in counts:
        counts['1'] = 0

    total_counts = counts['0'] + counts['1']
    z = counts['0'] - counts['1']
    z = z / total_counts
    
    return z

In [25]:
z = measure_z(Operational_circuit_try)
print("<Z> =", str(z))

<Z> = 1.0


# 2. X

In [28]:
def measure_x_circuit(given_circuit):
    x_meas = given_circuit.copy()
    x_meas.h(0)
    x_meas.measure_all()

    return x_meas

In [29]:
x_meas = measure_x_circuit(Operational_circuit_try)
x_meas.draw()

┌───────┐┌───┐ ░ ┌─┐
   q_0: ┤ RY(0) ├┤ H ├─░─┤M├
        └───────┘└───┘ ░ └╥┘
meas_0: ══════════════════╩═

In [32]:
def measure_x(given_circuit, num_shots = 10000):
    
    x_meas = measure_x_circuit(given_circuit)
    
    result = execute(x_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(x_meas)

    if '0' not in counts:
        counts['0'] = 0
    if '1' not in counts:
        counts['1'] = 0

    total_counts = counts['0'] + counts['1'] 
    x = counts['0'] - counts['1']
    x = x / total_counts
    
    return x

In [33]:
x = measure_x(Operational_circuit_try)
print("<X> =", str(x))

<X> = -0.0182


# 3. Y

In [34]:
def measure_y_circuit(given_circuit):
    y_meas = given_circuit.copy()
    y_meas.rz(-pi/2,0)
    y_meas.h(0)

    y_meas.measure_all()

    return y_meas

In [35]:
y_meas = measure_y_circuit(Operational_circuit_try)
y_meas.draw()

┌───────┐┌───────────┐┌───┐ ░ ┌─┐
   q_0: ┤ RY(0) ├┤ RZ(-pi/2) ├┤ H ├─░─┤M├
        └───────┘└───────────┘└───┘ ░ └╥┘
meas_0: ═══════════════════════════════╩═

In [36]:
def measure_y(given_circuit, num_shots = 10000):
    
    yy_meas = measure_y_circuit(given_circuit)
    
    result = execute(y_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(y_meas)

    if '0' not in counts:
        counts['00'] = 0
    if '1' not in counts:
        counts['01'] = 0

    total_counts = counts['0'] + counts['1'] 
    y = counts['0']  - counts['1']
    y = y / total_counts
    
    return y

In [37]:
y = measure_y(Operational_circuit_try)
print("<y> =", str(y))

<y> = 0.0148


# Matrix and Decomposition

In [345]:
M= np.matrix('-0.5 0.5  ;0.5 -0.5 ')

In [346]:
coeffs, obs_list = qml.utils.decompose_hamiltonian(M)
H = qml.Hamiltonian(coeffs, obs_list)

In [347]:
print(H)

(-0.5) [I0]
+ (0.5) [X0]


In [348]:
def coef(H):
    coeff = [0,0,0,0]
    h = str(H)
    for i in range (len(h)):
        p = ""
        if h[i] =="(":
            j = i+1 
            while h[j]!=")" :
                p=p+h[j]
                j+=1
            
            if h[j+3] == "I":
                coeff[0] = float(p)
                
            if h[j+3] == "X":
                coeff[1] = float(p)
                
            if h[j+3] == "Y":
                coeff[2] = float(p)
                
            if h[j+3] == "Z":
                coeff[3] = float(p)
    
    return(coeff)
            
            

# Cost_Function

In [349]:
def eigen_value(params):
    num_shots = 10000
    z = measure_z(Operational_circuit(params), num_shots = num_shots)
    x = measure_x(Operational_circuit(params), num_shots = num_shots)
    y = measure_y(Operational_circuit(params), num_shots = num_shots)
    coeff =coef(H) 
    ev = coeff[0]+coeff[1]*x+coeff[2]*y+coeff[3]*z
    
    return ev

In [350]:
params = [0]

In [351]:
ev = eigen_value(params)     #Try energy with params = [0,0,0,0,0,0]
print("The Eigen_Value of the trial state is", str(ev))

The Eigen_Value of the trial state is -0.4881


# Finding Lowest Eignevalue

In [352]:
from qiskit.aqua.components.optimizers import COBYLA,SLSQP,SPSA

optimizer = COBYLA(50,0.001)

In [353]:
ret = optimizer.optimize(num_vars=1, objective_function=eigen_value, initial_point=params)

In [354]:
eigen_value(ret[0])

-1.0

# Eigen State.

In [355]:
backend_state = Aer.get_backend('statevector_simulator')

In [356]:

result = execute(Operational_circuit(ret[0]),backend_state).result()
out_state = result.get_statevector()


In [357]:
print(out_state)

[ 0.71003388+0.j -0.70416751+0.j]


# Check

In [358]:
qc = QuantumCircuit(1)
qc.initialize(out_state,0)

In [359]:
def eigen_value_state(qc):
    num_shots = 100000
    z = measure_z(qc, num_shots = num_shots)
    x = measure_x(qc, num_shots = num_shots)
    y = measure_y(qc, num_shots = num_shots)
    coeff =coef(H) 
    ev = coeff[0]+coeff[1]*x+coeff[2]*y+coeff[3]*z
    
    return ev

In [360]:
 eigen_value_state(qc)

-0.99997

# Exponential Matrix 

In [361]:
from scipy.linalg import expm, sinm, cosm

In [414]:
M= np.matrix('0j 1j  ;1j 0j ')
M1= np.matrix('0j -1j  ;-1j 0j ')
expm(M)

array([[0.54030231+0.j        , 0.        +0.84147098j],
       [0.        +0.84147098j, 0.54030231+0.j        ]])

In [415]:
def mult(X,Y):
    Z = [0,0,0,0]
    Z[0] = round(expm(X)[0][0]*expm(Y)[0][0]+expm(X)[0][1]*expm(Y)[1][0])
    Z[1] = round(expm(X)[0][0]*expm(Y)[0][1]+expm(X)[0][1]*expm(Y)[1][1])
    Z[2] = round(expm(X)[1][0]*expm(Y)[0][0]+expm(X)[1][1]*expm(Y)[1][0])
    Z[3] = round(expm(X)[1][0]*expm(Y)[0][1]+expm(X)[1][1]*expm(Y)[1][1])
    return(Z)

In [416]:
mult(M,M1)

[(1+0j), -0j, 0j, (1+0j)]

In [417]:
expm(M1)

array([[0.54030231+0.j        , 0.        -0.84147098j],
       [0.        -0.84147098j, 0.54030231+0.j        ]])